In [11]:
from my_functions import create_corpus_file, process_dataset, reddit_data_cleaning, train_word2vec, load_thesis_datasets,\
generate_word2vector, generate_word_embeddings, setup_rnn, generate_rnn_model, dataset_shuffler

import logging
import time
import string
import sys
import re
import os
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding, LeakyReLU
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score

In [2]:
df = process_dataset(load_thesis_datasets(), '2', '2-cleaned')

/home/opc/my_functions.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  data[cleaned_col] = data[cleaned_col].str.replace(pat, '')
/home/opc/my_functions.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  data[cleaned_col] = data[cleaned_col].str.replace(r'\s+', ' ')


Processing completed in: 8 seconds.


In [3]:
df['tokens']

0     [attempt, make, devices, shown, video, informa...
1     [oh, word, really, need, another, two, three, ...
2     [hi, thanks, modern, gadgets, every, person, s...
3     [hello, friends, video, show, heat, camping, t...
4     [roundly, hated, widely, loved, ar15, rifle, s...
                            ...                        
92    [music, history, tells, us, adolf, hitler, dem...
93    [folks, turbo, demand, truth, im, going, take,...
94    [hey, everybody, welcome, back, listen, know, ...
95    [hey, today, november, 23rd, 2020, im, federal...
96    [ig12com, back, another, video, blog, one, wel...
Name: tokens, Length: 578, dtype: object

In [17]:
# Customized version of our word2vec embedding creation function

def get_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings100(vectors, clean_comments, generate_missing=False, k=100):
    k = k
    embeddings = clean_comments.apply(lambda x: get_word2vec(x, vectors, generate_missing=generate_missing, k=100))
    return list(embeddings)

def get_word2vec_embeddings200(vectors, clean_comments, generate_missing=False, k=200):
    k = k
    embeddings = clean_comments.apply(lambda x: get_word2vec(x, vectors, generate_missing=generate_missing, k=200))
    return list(embeddings)

def get_word2vec_embeddings300(vectors, clean_comments, generate_missing=False, k=300):
    k = k
    embeddings = clean_comments.apply(lambda x: get_word2vec(x, vectors, generate_missing=generate_missing, k=300))
    return list(embeddings)

In [10]:
# Load the word2vec models we have trained

wikicorpus = 'glove.6B.200d.txt'


reddit_sg_dim100_win10 = 'reddit_model_SG_dim100_con10.txt'
reddit_sg_dim200_win3 = 'reddit_model_SG_dim200_win3.txt'
reddit_sg_dim200_win5 = 'reddit_model_SG_dim200_win5.txt'
reddit_sg_dim200_win10 = 'reddit_sg_dim300_win5.txt'
reddit_sg_dim300_win5 = 'reddit_sg_dim300_win5.txt'
reddit_sg_dim300_win10 = 'reddit_model_SG_dim300_win10.txt'
reddit_cbow_dim100_win5 = 'reddit_model_CBOW_dim100_con5.txt'
reddit_cbow_dim100_win10 = 'reddit_model_CBOW_dim200_con10.txt'
reddit_cbow_dim200_win5 = 'reddit_CBOW_DIM200_win5.txt'
reddit_cbow_dim200_win10 = 'reddit_model_CBOW_dim200_con10.txt'
reddit_bow_dim300_win10 = 'reddit_model_CBOW_dim300_con10.txt'

transcript_sg_dim100_win5 = 'transcript_SG_DIM100_win5.txt'
transcript_sg_dim100_win10 = 'transcript_SG_DIM100_win10.txt'
transcript_sg_dim200_win5 = 'transcript_SG_DIM200_win5.txt'
transcript_sg_dim200_win10 = 'transcript_SG_DIM200_win10.txt'
transcript_sg_dim300_win5 = 'transcript_SG_DIM300_WIN5.txt'
transcript_sg_dim300_win10 = 'transcript_SG_DIM300_WIN10.txt'
transcript_cbow_dim100_win5 = 'transcript_CBOW_DIM100_win5.txt'
transcript_cbow_dim100_win10 = 'transcript_CBOW_DIM100_win10.txt'
transcript_cbow_dim200_win5 = 'transcript_CBOW_DIM200_win5.txt'
transcript_cbow_dim200_win10 = 'transcript_CBOW_DIM200_win10.txt'
transcript_cbow_dim300_win5 = 'transcript_CBOW_DIM300_win5.txt'
transcript_cbow_dim300_win10 = 'transcript_CBOW_DIM300_win10.txt'

In [23]:
# Load models and retrieve word vectors

# REDDIT SKIP-GRAM MODELS

# reddit_sg_dim100_win10_model = KeyedVectors.load(reddit_sg_dim100_win10)
# reddit_sg_dim100_win10_vectors = reddit_sg_dim100_win10_model.wv

# reddit_sg_dim200_win5_model = KeyedVectors.load(reddit_sg_dim200_win5)
# reddit_sg_dim200_win5_vectors = reddit_sg_dim200_win5_model.wv

# reddit_sg_dim200_win10_model = KeyedVectors.load(reddit_sg_dim200_win10)
# reddit_sg_dim200_win10_vectors = reddit_sg_dim200_win10_model.wv

# reddit_sg_dim300_win5_model = KeyedVectors.load(reddit_sg_dim300_win5)
# reddit_sg_dim300_win5_vectors = reddit_sg_dim300_win5_model.wv

# reddit_sg_dim300_win10_model = KeyedVectors.load(reddit_sg_dim300_win10) #NOT WORKING
# reddit_sg_dim300_win10_vectors = reddit_sg_dim300_win10_model.wv

# reddit_sg_dim100_win10_model = KeyedVectors.load(reddit_sg_dim300_win5)
# reddit_sg_dim100_win10_vectors = reddit_sg_dim300_win5_model.wv

# REDDIT CBOW MODELS

# reddit_cbow_dim100_win5_model = KeyedVectors.load(reddit_cbow_dim100_win5)
# reddit_cbow_dim100_win5_vectors = reddit_cbow_dim100_win5_model.wv

# reddit_cbow_dim100_win10_model = KeyedVectors.load(reddit_cbow_dim100_win10)
# reddit_cbow_dim100_win10_vectors = reddit_cbow_dim100_win10_model.wv

# reddit_cbow_dim200_win5_model = KeyedVectors.load(reddit_cbow_dim200_win5)
# reddit_cbow_dim200_win5_vectors = reddit_cbow_dim200_win5_model.wv

# reddit_cbow_dim200_win10_model = KeyedVectors.load(reddit_cbow_dim200_win10)
# reddit_cbow_dim200_win10_vectors = reddit_cbow_dim200_win10_model.wv

# reddit_bow_dim300_win10_model = KeyedVectors.load(reddit_bow_dim300_win10)
# reddit_bow_dim300_win10_vectors = reddit_bow_dim300_win10_model.wv

# TRANSCRIPT SKIP-GRAM MODELS

# transcript_sg_dim100_win5_model = KeyedVectors.load(transcript_sg_dim100_win5)
# transcript_sg_dim100_win5_vectors = transcript_sg_dim100_win5_model.wv

# transcript_sg_dim100_win10_model = KeyedVectors.load(transcript_sg_dim100_win10)
# transcript_sg_dim100_win10_vectors = transcript_sg_dim100_win10_model.wv

# transcript_sg_dim200_win5_model = KeyedVectors.load(transcript_sg_dim200_win5)
# transcript_sg_dim200_win5_vectors = transcript_sg_dim200_win5_model.wv

# transcript_sg_dim200_win10_model = KeyedVectors.load(transcript_sg_dim200_win10)
# transcript_sg_dim200_win10_vectors = transcript_sg_dim200_win10_model.wv

# transcript_sg_dim300_win5_model = KeyedVectors.load(transcript_sg_dim300_win5)
# transcript_sg_dim300_win5_vectors = transcript_sg_dim300_win5_model.wv

# transcript_sg_dim300_win10_model = KeyedVectors.load(transcript_sg_dim300_win10)
# transcript_sg_dim300_win10_vectors = transcript_sg_dim300_win10_model.wv

# TRANSCRIPT CBOW MODELS

# transcript_cbow_dim100_win5_model = KeyedVectors.load(transcript_cbow_dim100_win5)
# transcript_cbow_dim100_win5_vectors = transcript_cbow_dim100_win5_model.wv

# transcript_cbow_dim100_win10_model = KeyedVectors.load(transcript_cbow_dim100_win10)
# transcript_cbow_dim100_win10_vectors = transcript_cbow_dim100_win10_model.wv

# transcript_cbow_dim200_win5_model = KeyedVectors.load(transcript_cbow_dim200_win5)
# transcript_cbow_dim200_win5_vectors = transcript_cbow_dim200_win5_model.wv

# transcript_cbow_dim200_win10_model = KeyedVectors.load(transcript_cbow_dim200_win10)
# transcript_cbow_dim200_win10_vectors = transcript_cbow_dim200_win10_model.wv

# transcript_cbow_dim300_win5_model = KeyedVectors.load(transcript_cbow_dim300_win5)
# transcript_cbow_dim300_win5_vectors = transcript_cbow_dim300_win5_model.wv

transcript_cbow_dim300_win10_model = KeyedVectors.load(transcript_cbow_dim300_win10)
transcript_cbow_dim300_win10_vectors = transcript_cbow_dim300_win10_model.wv





In [7]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(df['tokens'],df['3'],test_size=0.2)

In [8]:
# Load WikiCorpus
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(wikicorpus, tmp_file)
wikicorpus_model = KeyedVectors.load_word2vec_format(tmp_file)

# # # LOAD WIKICORPUS TRAINED WORD VECTORS

# wikicorpus_training_embeddings = get_word2vec_embeddings200(wikicorpus_model, Train_X, generate_missing=True) 
# wikicorpus_test_embeddings = get_word2vec_embeddings200(wikicorpus_model, Test_X, generate_missing=True)


/home/opc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# LOAD WIKICORPUS TRAINED WORD VECTORS

wikicorpus_training_embeddings = get_word2vec_embeddings200(wikicorpus_model, Train_X, generate_missing=True) 
wikicorpus_test_embeddings = get_word2vec_embeddings200(wikicorpus_model, Test_X, generate_missing=True)


In [24]:
# LOAD REDDIT SKIP-GRAM TRAINED WORD VECTORS

# reddit_sg_dim100_win10_training_embeddings = get_word2vec_embeddings100(transcript_sg_dim100_win10_vectors, Train_X, generate_missing=True) 
# reddit_sg_dim100_win10_test_embeddings = get_word2vec_embeddings100(reddit_sg_dim100_win10_vectors, Test_X, generate_missing=True)

# reddit_sg_dim200_win5_training_embeddings = get_word2vec_embeddings200(reddit_sg_dim200_win5_vectors, Train_X, generate_missing=True, k=200) 
# reddit_sg_dim200_win5_test_embeddings = get_word2vec_embeddings200(reddit_sg_dim200_win5_vectors, Test_X, generate_missing=True, k=200)

# reddit_sg_dim200_win10_training_embeddings = get_word2vec_embeddings200(reddit_sg_dim200_win10_vectors, Train_X, generate_missing=True, k=200) 
# reddit_sg_dim200_win10_test_embeddings = get_word2vec_embeddings200(reddit_sg_dim200_win10_vectors, Test_X, generate_missing=True, k=200)

# reddit_sg_dim300_win5_training_embeddings = get_word2vec_embeddings300(reddit_sg_dim300_win5_vectors, Train_X, generate_missing=True, k=300) 
# reddit_sg_dim300_win5_test_embeddings = get_word2vec_embeddings300(reddit_sg_dim300_win5_vectors, Test_X, generate_missing=True, k=300)

# reddit_sg_dim300_win10_training_embeddings = get_word2vec_embeddings300(reddit_sg_dim300_win10_vectors, Train_X, generate_missing=True, k=300) 
# reddit_sg_dim300_win10_test_embeddings = get_word2vec_embeddings300(reddit_sg_dim300_win10_vectors, Test_X, generate_missing=True, k=300)

# # # LOAD REDDIT CBOW TRAINED WORD VECTORS

# reddit_bow_dim100_win5_training_embeddings = get_word2vec_embeddings100(reddit_cbow_dim100_win5_vectors, Train_X, generate_missing=True, k=100) 
# reddit_bow_dim100_win5_test_embeddings = get_word2vec_embeddings100(reddit_cbow_dim100_win5_vectors, Test_X, generate_missing=True, k=100)

# reddit_bow_dim100_win10_training_embeddings = get_word2vec_embeddings100(reddit_cbow_dim100_win10_vectors, Train_X, generate_missing=True, k=100) 
# reddit_bow_dim100_win10_test_embeddings = get_word2vec_embeddings100(reddit_cbow_dim100_win10_vectors, Test_X, generate_missing=True, k=100)

# reddit_bow_dim200_win5_training_embeddings = get_word2vec_embeddings200(reddit_cbow_dim200_win5_vectors, Train_X, generate_missing=True, k=200) 
# reddit_bow_dim200_win5_test_embeddings = get_word2vec_embeddings200(reddit_cbow_dim200_win5_vectors, Test_X, generate_missing=True, k=200)

# reddit_bow_dim200_win10_training_embeddings = get_word2vec_embeddings200(reddit_cbow_dim200_win10_vectors, Train_X, generate_missing=True, k=200) 
# reddit_bow_dim200_win10_test_embeddings = get_word2vec_embeddings200(reddit_cbow_dim200_win10_vectors, Test_X, generate_missing=True, k=200)

# reddit_bow_dim300_win10_training_embeddings = get_word2vec_embeddings300(reddit_bow_dim300_win10_vectors, Train_X, generate_missing=True, k=300) 
# reddit_bow_dim300_win10_test_embeddings = get_word2vec_embeddings300(reddit_bow_dim300_win10_vectors, Test_X, generate_missing=True, k=300)

# # # LOAD TRANSCRIPT SKIP-GRAM TRAINED WORD VECTORS

# transcript_sg_dim100_win5_training_embeddings = get_word2vec_embeddings100(transcript_sg_dim100_win5_vectors, Train_X, generate_missing=True, k=100) 
# transcript_sg_dim100_win5_test_embeddings = get_word2vec_embeddings100(transcript_sg_dim100_win5_vectors, Test_X, generate_missing=True, k=100)

# transcript_sg_dim100_win10_training_embeddings = get_word2vec_embeddings100(transcript_sg_dim100_win10_vectors, Train_X, generate_missing=True, k=100) 
# transcript_sg_dim100_win10_test_embeddings = get_word2vec_embeddings100(transcript_sg_dim100_win10_vectors, Test_X, generate_missing=True, k=100)

# transcript_sg_dim200_win5_training_embeddings = get_word2vec_embeddings200(transcript_sg_dim200_win5_vectors, Train_X, generate_missing=True, k=200) 
# transcript_sg_dim200_win5_test_embeddings = get_word2vec_embeddings200(transcript_sg_dim200_win5_vectors, Test_X, generate_missing=True, k=200)

# transcript_sg_dim200_win10_training_embeddings = get_word2vec_embeddings200(transcript_sg_dim200_win10_vectors, Train_X, generate_missing=True, k=200) 
# transcript_sg_dim200_win10_test_embeddings = get_word2vec_embeddings200(transcript_sg_dim200_win10_vectors, Test_X, generate_missing=True, k=200)

# transcript_sg_dim300_win5_training_embeddings = get_word2vec_embeddings300(transcript_sg_dim300_win5_vectors, Train_X, generate_missing=True, k=300) 
# transcript_sg_dim300_win5_test_embeddings = get_word2vec_embeddings300(transcript_sg_dim300_win5_vectors, Test_X, generate_missing=True, k=300)

# transcript_sg_dim300_win10_training_embeddings = get_word2vec_embeddings300(transcript_sg_dim300_win10_vectors, Train_X, generate_missing=True, k=300) 
# transcript_sg_dim300_win10_test_embeddings = get_word2vec_embeddings300(transcript_sg_dim300_win10_vectors, Test_X, generate_missing=True, k=300)

# # # LOAD TRANSCRIPT CBOW TRAINED WORD VECTORS

# transcript_cbow_dim100_win5_training_embeddings = get_word2vec_embeddings100(transcript_cbow_dim100_win5_vectors, Train_X, generate_missing=True, k=100) 
# transcript_cbow_dim100_win5_test_embeddings = get_word2vec_embeddings100(transcript_cbow_dim100_win5_vectors, Test_X, generate_missing=True, k=100)

# transcript_cbow_dim100_win10_training_embeddings = get_word2vec_embeddings100(transcript_cbow_dim100_win10_vectors, Train_X, generate_missing=True, k=100) 
# transcript_cbow_dim100_win10_test_embeddings = get_word2vec_embeddings100(transcript_cbow_dim100_win10_vectors, Test_X, generate_missing=True, k=100)

# transcript_cbow_dim200_win5_training_embeddings = get_word2vec_embeddings200(transcript_cbow_dim200_win5_vectors, Train_X, generate_missing=True, k=200) 
# transcript_cbow_dim200_win5_test_embeddings = get_word2vec_embeddings200(transcript_cbow_dim200_win5_vectors, Test_X, generate_missing=True, k=200)

# transcript_cbow_dim200_win10_training_embeddings = get_word2vec_embeddings200(transcript_cbow_dim200_win10_vectors, Train_X, generate_missing=True, k=200) 
# transcript_cbow_dim200_win10_test_embeddings = get_word2vec_embeddings200(transcript_cbow_dim200_win10_vectors, Test_X, generate_missing=True, k=200)

# transcript_cbow_dim300_win5_training_embeddings = get_word2vec_embeddings300(transcript_cbow_dim300_win5_vectors, Train_X, generate_missing=True, k=300) 
# transcript_cbow_dim300_win5_test_embeddings = get_word2vec_embeddings300(transcript_cbow_dim300_win5_vectors, Test_X, generate_missing=True, k=300)

transcript_cbow_dim300_win10_training_embeddings = get_word2vec_embeddings300(transcript_cbow_dim300_win10_vectors, Train_X, generate_missing=True, k=300) 
transcript_cbow_dim300_win10_test_embeddings = get_word2vec_embeddings300(transcript_cbow_dim300_win10_vectors, Test_X, generate_missing=True, k=300)



In [ ]:
# Load WikiCorpus
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(wikicorpus, tmp_file)
wikicorpus_model = KeyedVectors.load_word2vec_format(tmp_file)


In [19]:
# Encoding
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# REDDIT SKIP-GRAM WORD VECTORS TESTED AGAINST SVM

In [20]:
# Hyperparameter setting
C = 0.75

In [ ]:
# # Preparing our SVM Classifier for tuning
# # fit the training dataset on the classifier
# SVM = svm.SVC(C=1.1, kernel='linear', degree=10, gamma='auto')
# SVM.fit(reddit_sg_dim100_win10_training_embeddings,Train_Y)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict(reddit_sg_dim100_win10_test_embeddings)
# # Use accuracy_score function to get the accuracy
# print("Model:", 'reddit_sg_dim100_win10', 'results...')
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)


In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(reddit_sg_dim200_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(reddit_sg_dim200_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'reddit_sg_dim200_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# # Preparing our SVM Classifier for tuning
# # fit the training dataset on the classifier
# SVM = svm.SVC(C=1, kernel='linear', degree=10, gamma='auto')
# SVM.fit(reddit_sg_dim200_win10_training_embeddings,Train_Y)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict(reddit_sg_dim200_win10_test_embeddings)
# # Use accuracy_score function to get the accuracy
# print("Model:", 'reddit_sg_dim200_win10', 'results...')
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(reddit_sg_dim300_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(reddit_sg_dim300_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'reddit_sg_dim300_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(reddit_sg_dim300_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(reddit_sg_dim300_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'reddit_sg_dim300_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

# REDDIT CBOW WORD VECTORS TESTED AGAINST SVM

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(reddit_bow_dim100_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(reddit_bow_dim100_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'reddit_bow_dim100_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# # Preparing our SVM Classifier for tuning
# # fit the training dataset on the classifier
# SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
# SVM.fit(reddit_bow_dim100_win10_training_embeddings,Train_Y)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict(reddit_bow_dim100_win10_test_embeddings)
# # Use accuracy_score function to get the accuracy
# print("Model:", 'reddit_bow_dim100_win10', 'results...')
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
# # Preparing our SVM Classifier for tuning
# # fit the training dataset on the classifier
# SVM = svm.SVC(C=1, kernel='linear', degree=10, gamma='auto')
# SVM.fit(reddit_bow_dim200_win5_training_embeddings,Train_Y)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict(reddit_bow_dim200_win5_test_embeddings)
# # Use accuracy_score function to get the accuracy
# print("Model:", 'reddit_bow_dim200_win5', 'results...')
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(reddit_bow_dim200_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(reddit_bow_dim200_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'reddit_bow_dim200_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(reddit_bow_dim300_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(reddit_bow_dim300_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'reddit_bow_dim300_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

# TRANSCRIPT SKIP-GRAM WORD VECTORS TESTED AGAINST SVM

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_sg_dim100_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_sg_dim100_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_sg_dim100_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_sg_dim100_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_sg_dim100_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_sg_dim100_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# # Preparing our SVM Classifier for tuning
# # fit the training dataset on the classifier
# SVM = svm.SVC(C=1, kernel='linear', degree=10, gamma='auto')
# SVM.fit(transcript_sg_dim200_win5_training_embeddings,Train_Y)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict(transcript_sg_dim200_win5_test_embeddings)
# # Use accuracy_score function to get the accuracy
# print("Model:", 'transcript_sg_dim200_win5', 'results...')
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
# # Preparing our SVM Classifier for tuning
# # fit the training dataset on the classifier
# SVM = svm.SVC(C=1, kernel='linear', degree=10, gamma='auto')
# SVM.fit(transcript_sg_dim200_win10_training_embeddings,Train_Y)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict(transcript_sg_dim200_win10_test_embeddings)
# # Use accuracy_score function to get the accuracy
# print("Model:", 'transcript_sg_dim200_win10', 'results...')
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_sg_dim300_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_sg_dim300_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_sg_dim300_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_sg_dim300_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_sg_dim300_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_sg_dim300_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

# TRASCRIPT CBOW WORD VECTORS TESTED AGAINST SVM

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_cbow_dim100_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_cbow_dim100_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_cbow_dim100_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_cbow_dim100_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_cbow_dim100_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_cbow_dim100_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_cbow_dim200_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_cbow_dim200_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_cbow_dim200_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_cbow_dim200_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_cbow_dim200_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_cbow_dim200_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [ ]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_cbow_dim300_win5_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_cbow_dim300_win5_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_cbow_dim300_win5', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

In [25]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=1, kernel='linear', degree=10, gamma='auto')
SVM.fit(transcript_cbow_dim300_win10_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(transcript_cbow_dim300_win10_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'transcript_cbow_dim300_win10', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

Model: transcript_cbow_dim300_win10 results...
SVM Accuracy Score ->  87.93103448275862


# WIKICORPUS WORD VECTORS TESTED AGAINST SVM

In [25]:
C = 1.25

In [30]:
# Preparing our SVM Classifier for tuning
# fit the training dataset on the classifier
SVM = svm.SVC(C=C, kernel='linear', degree=10, gamma='auto')
SVM.fit(wikicorpus_training_embeddings,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(wikicorpus_test_embeddings)
# Use accuracy_score function to get the accuracy
print("Model:", 'WikiCorpus', 'results...')
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("F1-Score ->", f1_score(predictions_SVM, Test_Y, average='macro'))

Model: WikiCorpus results...
SVM Accuracy Score ->  81.89655172413794
F1-Score -> 0.8139747995418098
